## Load Data

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SQLContext

In [ ]:
sc = SparkContext()
spark = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=yarn) created by <module> at /opt/conda/anaconda/lib/python3.7/site-packages/IPython/utils/py3compat.py:188 

In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


In [ ]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Initialize a client
storage_client = storage.Client()

# Define your bucket name and object name
bucket_name = 'ds5460-final-bucket'
blob_name = 'notebooks/jupyter/Master Files/master_listings.csv'

# Create a bucket object
bucket = storage_client.bucket(bucket_name)

# Create a blob object
blob = bucket.blob(blob_name)

# Download the content of the blob
byte_stream = BytesIO()
blob.download_to_file(byte_stream)
byte_stream.seek(0)

# Use Pandas to read the CSV from bytes
df = pd.read_csv(byte_stream)


0

/opt/conda/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,28,29,68) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,state,city
0,109,https://www.airbnb.com/rooms/109,20231203160142,2023-12-05,city scrape,Condo in Culver City · 2 bedrooms · 2 beds · 2...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,521,https://www.airbnb.com/users/show/521,Paolo,2008-06-27,"San Francisco, CA","Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA ...",NaN,NaN,50%,f,https://a0.muscache.com/im/pictures/user/e704a...,https://a0.muscache.com/im/pictures/user/e704a...,Culver City,1.0,3.0,"['email', 'phone']",t,t,NaN,Culver City,Other Cities,33.98301,-118.38607,Entire condo,Entire home/apt,6,NaN,2 baths,NaN,2.0,"[""Private patio or balcony"", ""Shared pool"", ""E...",$129.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,0,0,0,180,2023-12-05,2,0,0,2011-08-15,2016-05-15,4.00,5.00,5.00,3.00,4.00,5.00,4.00,NaN,f,1,1,0,0,0.01,CA,Los Angeles
1,2708,https://www.airbnb.com/rooms/2708,20231203160142,2023-12-04,city scrape,Rental unit in Los Angeles · ★4.85 · 1 bedroom...,NaN,"Walk and run to Runyon Canyon, it is open!<br ...",https://a0.muscache.com/pictures/miso/Hosting-...,3008,https://www.airbnb.com/users/show/3008,Chas.,2008-09-16,"Los Angeles, CA",Professional and technical writer.\nLiterary C...,NaN,NaN,100%,t,https://a0.muscache.com/im/pictures/user/d17cf...,https://a0.muscache.com/im/pictures/user/d17cf...,Hollywood,2.0,3.0,"['email', 'phone']",t,t,"Los Angeles, California, United States",Hollywood,City of Los Angeles,34.09625,-118.34605,Private room in rental unit,Private room,1,NaN,1 shared bath,NaN,1.0,"[""Wen, V05, Prell, among others. conditioner""...",$94.00,30,550,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,0,0,0,271,2023-12-04,40,2,0,2014-06-09,2023-07-31,4.85,4.80,4.93,5.00,4.98,4.95,4.88,NaN,t,2,0,2,0,0.35,CA,Los Angeles
2,2732,https://www.airbnb.com/rooms/2732,20231203160142,2023-12-04,city scrape,Rental unit in Santa Monica · ★4.41 · 1 bedroo...,NaN,"This is the best part of Santa Monica. Quiet, ...",https://a0.muscache.com/pictures/1082974/0f74c...,3041,https://www.airbnb.com/users/show/3041,Yoga Priestess,2008-09-17,"Santa Monica, CA",I have been teaching yoga and meditation for 3...,within an hour,100%,42%,f,https://a0.muscache.com/im/users/3041/profile_...,https://a0.muscache.com/im/users/3041/profile_...,Santa Monica,2.0,2.0,"['email', 'phone']",t,t,"Santa Monica, California, United States",Santa Monica,Other Cities,34.00440,-118.48095,Private room in rental unit,Private room,1,NaN,1 private bath,NaN,1.0,"[""Private patio or balcony"", ""Hair dryer"", ""De...",$179.00,7,27,7.0,7.0,27.0,27.0,7.0,27.0,NaN,t,25,55,85,360,2023-12-04,24,0,0,2011-06-06,2022-08-21,4.41,4.26,4.58,4.39,4.48,4.91,4.22,228269,f,2,1,1,0,0.16,CA,Los Angeles
3,2864,https://www

In [ ]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [ ]:
df.shape

(212992, 77)

## Features

make categories for the following features:

1,'property_type'

*   **apartment** if property_type is Apartment, Serviced apartment, Loft.
*   **hotel** if property_type is Bed & Breakfast, Boutique hotel, Hostel.
*   **condo** if property_type is Townhouse, Condominium.
*   **house** if property_type is Bungalow, House.
*   **other**, otherwise

2, 'room_type': ['Entire home/apt' 'Private room' 'Shared room' 'Hotel room']

3, 'accommodates'

4, 'bathrooms'. /'bathrooms_text'

5, 'bedrooms', 'beds'

6, 'amenities'

7, 'price': divides data into equal-sized bins based on quantiles for every state&city : ['Low', 'Medium', 'High', 'Very High']

8, 'number_of_reviews'

9, 'review_scores_rating'


All the features we plan to use/transform in the below analysis:
'property_type',
'room_type',
'price',
 'accommodates',
'bathrooms', 'bathrooms_text'
'bedrooms', 'beds'
 'number_of_reviews',
 'latitude',
'longitude'

New Feature: Distance to Downtown Calculation

We plan to utilize **latitude** and **longitude** coordinates to accurately calculate the distance from each city to its downtown area.

'property_type', 'room_type' and 'price' -> Apply one-hot encoding


In [ ]:
df = df[['id','latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'price', 'number_of_reviews','review_scores_rating','state', 'city']]

In [ ]:
df.head()

,id,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,number_of_reviews,review_scores_rating,state,city
0,109,33.98301,-118.38607,Entire condo,Entire home/apt,6,NaN,2 baths,NaN,2.0,$129.00,2,4.00,CA,Los Angeles
1,2708,34.09625,-118.34605,Private room in rental unit,Private room,1,NaN,1 shared bath,NaN,1.0,$94.00,40,4.85,CA,Los Angeles
2,2732,34.00440,-118.48095,Private room in rental unit,Private room,1,NaN,1 private bath,NaN,1.0,$179.00,24,4.41,CA,Los Angeles
3,2864,33.87490,-118.11515,Entire rental unit,Entire home/apt,2,NaN,1 bath,NaN,1.0,$50.00,0,NaN,CA,Los Angeles
4,63416,34.11344,-118.32399,Private room in home,Private room,3,NaN,1 private bath,NaN,2.0,$220.00,161,4.99,CA,Los Angeles


In [ ]:
df.isnull().sum()

id                           0
latitude                     0
longitude                    0
property_type                0
room_type                    0
accommodates                 0
bathrooms               187764
bathrooms_text             131
bedrooms                178857
beds                     17040
price                    22911
number_of_reviews            0
review_scores_rating     50321
state                        0
city                         0
dtype: int64

In [ ]:
df.columns

Index(['id', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
       'price', 'number_of_reviews', 'review_scores_rating', 'state', 'city'],
      dtype='object')

In [ ]:
import numpy as np
# Step 1: Handle 'half-bath' cases by replacing them with "0.5"
df['bathrooms_text'] = df['bathrooms_text'].replace(['Private half-bath', 'Half-bath','Shared half-bath'], '0.5 bath')

# Step 2: Filter rows where both bathrooms and bathrooms_text are null
df_filtered = df.dropna(subset=['bathrooms', 'bathrooms_text'], how='all')

# Step 3: Extract numeric value from 'bathrooms_text'
# This regex pattern looks for a sequence of digits possibly followed by a dot and more digits
df_filtered['bathrooms_text_numeric'] = df_filtered['bathrooms_text'].str.extract(r'(\d+\.?\d*)').astype(float)

# Step 4: Combine 'bathrooms' and 'bathrooms_text_numeric' into one column
# np.where(condition, [x, y]) is used here: if condition is true it takes x, otherwise y.
df_filtered['bathrooms_combined'] = np.where(df_filtered['bathrooms'].notnull(), df_filtered['bathrooms'], df_filtered['bathrooms_text_numeric'])

df_filtered.head()

/opt/conda/anaconda/bin/ipython:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])
/opt/conda/anaconda/bin/ipython:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,price,number_of_reviews,review_scores_rating,state,city,bathrooms_text_numeric,bathrooms_combined
0,109,33.98301,-118.38607,Entire condo,Entire home/apt,6,NaN,2 baths,NaN,2.0,$129.00,2,4.00,CA,Los Angeles,2.0,2.0
1,2708,34.09625,-118.34605,Private room in rental unit,Private room,1,NaN,1 shared bath,NaN,1.0,$94.00,40,4.85,CA,Los Angeles,1.0,1.0
2,2732,34.00440,-118.48095,Private room in rental unit,Private room,1,NaN,1 private bath,NaN,1.0,$179.00,24,4.41,CA,Los Angeles,1.0,1.0
3,2864,33.87490,-118.11515,Entire rental unit,Entire home/apt,2,NaN,1 bath,NaN,1.0,$50.00,0,NaN,CA,Los Angeles,1.0,1.0
4,63416,34.11344,-118.32399,Private room in home,Private room,3,NaN,1 private bath,NaN,2.0,$220.00,161,4.99,CA,Los Angeles,1.0,1.0


In [ ]:
df_filtered.isnull().sum()

id                             0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                 187639
bathrooms_text                 6
bedrooms                  178761
beds                       17007
price                      22881
number_of_reviews              0
review_scores_rating       50261
state                          0
city                           0
bathrooms_text_numeric         6
bathrooms_combined             0
dtype: int64

In [ ]:
df_filtered= df_filtered[['id','latitude', 'longitude', 'property_type', 'room_type','bathrooms_combined', 'accommodates', 'beds', 'price',
       'number_of_reviews', 'review_scores_rating','state', 'city']]

In [ ]:
df_filtered.isnull().sum()

id                          0
latitude                    0
longitude                   0
property_type               0
room_type                   0
bathrooms_combined          0
accommodates                0
beds                    17007
price                   22881
number_of_reviews           0
review_scores_rating    50261
state                       0
city                        0
dtype: int64

In [ ]:
# Remove rows where 'price' is NaN
df_filtered = df_filtered.dropna(subset=['price'])
df_filtered.isnull().sum()

id                          0
latitude                    0
longitude                   0
property_type               0
room_type                   0
bathrooms_combined          0
accommodates                0
beds                     2088
price                       0
number_of_reviews           0
review_scores_rating    42901
state                       0
city                        0
dtype: int64

In [ ]:
df_filtered.shape

(189986, 13)

**For some of the new hotels: number_of_reviews=0 & review_scores_rating=nan**

In [ ]:
unique_property_types = df_filtered['property_type'].unique()
print(f"Unique property types: {unique_property_types}")


Unique property types: ['Entire condo' 'Private room in rental unit' 'Entire rental unit'
 'Private room in home' 'Private room in tiny home' 'Entire guesthouse'
 'Entire bungalow' 'Private room in townhouse' 'Private room in loft'
 'Entire vacation home' 'Room in hotel' 'Entire home' 'Entire cottage'
 'Room in boutique hotel' 'Entire guest suite' 'Entire townhouse'
 'Private room in hostel' 'Private room' 'Private room in condo'
 'Camper/RV' 'Farm stay' 'Entire loft' 'Private room in guest suite'
 'Private room in farm stay' 'Shared room in home'
 'Private room in bungalow' 'Shared room in hostel' 'Shared room in villa'
 'Entire place' 'Entire villa' 'Private room in treehouse'
 'Private room in guesthouse' 'Room in aparthotel' 'Ranch' 'Treehouse'
 'Entire cabin' 'Shared room in rental unit' 'Private room in cottage'
 'Private room in villa' 'Private room in bed and breakfast' 'Yurt'
 'Entire serviced apartment' 'Tiny home' 'Shared room in condo'
 'Private room in serviced apartment' 

In [ ]:
unique_room_types = df_filtered['room_type'].unique()
print(f"Unique room types: {unique_room_types}")


Unique room types: ['Entire home/apt' 'Private room' 'Shared room' 'Hotel room']


property_category has the following values:

1, apartment if property_type is Apartment, Serviced apartment, Loft.

2, hotel if property_type is Bed & Breakfast, Boutique hotel, Hostel.

3, condo if property_type is Townhouse, Condominium.

4, house if property_type is Bungalow, House.

5, other, otherwise

In [ ]:
# Python adaptation for categorization
def categorize_property(property_type):
    property_type = property_type.lower()
    if any(keyword in property_type for keyword in ["apartment", "loft", "serviced apartment"]):
        return "apartment"
    elif any(keyword in property_type for keyword in ["bed & breakfast", "boutique hotel", "hostel", "hotel", "resort", "aparthotel", "bed and breakfast"]):
        return "hotel"
    elif any(keyword in property_type for keyword in ["townhouse", "condominium", "condo"]):
        return "condo"
    elif any(keyword in property_type for keyword in ["bungalow", "house", "villa", "cottage", "chalet", "in-law", "guesthouse", "cabin"]):
        return "house"
    else:
        return "other"

# Apply the function to the DataFrame
df_filtered['property_category'] = df_filtered['property_type'].apply(categorize_property)

# To check the distribution of the new categories
category_distribution = df_filtered['property_category'].value_counts()
print(category_distribution)


other        128919
condo         35903
house         11462
hotel          8064
apartment      5638
Name: property_category, dtype: int64


In [ ]:
df_filtered = df_filtered [['id', 'latitude', 'longitude', 'property_category', 'room_type', 'accommodates','bathrooms_combined', 'beds',  'price',
       'number_of_reviews', 'review_scores_rating','state', 'city']]

df_filtered.head()

,id,latitude,longitude,property_category,room_type,accommodates,bathrooms_combined,beds,price,number_of_reviews,review_scores_rating,state,city
0,109,33.98301,-118.38607,condo,Entire home/apt,6,2.0,2.0,$129.00,2,4.00,CA,Los Angeles
1,2708,34.09625,-118.34605,other,Private room,1,1.0,1.0,$94.00,40,4.85,CA,Los Angeles
2,2732,34.00440,-118.48095,other,Private room,1,1.0,1.0,$179.00,24,4.41,CA,Los Angeles
3,2864,33.87490,-118.11515,other,Entire home/apt,2,1.0,1.0,$50.00,0,NaN,CA,Los Angeles
4,63416,34.11344,-118.32399,other,Private room,3,1.0,2.0,$220.00,161,4.99,CA,Los Angeles


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189986 entries, 0 to 212991
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    189986 non-null  int64  
 1   latitude              189986 non-null  float64
 2   longitude             189986 non-null  float64
 3   property_category     189986 non-null  object 
 4   room_type             189986 non-null  object 
 5   accommodates          189986 non-null  int64  
 6   bathrooms_combined    189986 non-null  float64
 7   beds                  187898 non-null  float64
 8   price                 189986 non-null  object 
 9   number_of_reviews     189986 non-null  int64  
 10  review_scores_rating  147085 non-null  float64
 11  state                 189986 non-null  object 
 12  city                  189986 non-null  object 
dtypes: float64(5), int64(3), object(5)
memory usage: 20.3+ MB


In [ ]:
# Convert price from string to float
df_filtered['price'] = df_filtered['price'].str.replace('$', '').str.replace(',', '').astype(float)



/opt/conda/anaconda/bin/ipython:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189986 entries, 0 to 212991
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    189986 non-null  int64  
 1   latitude              189986 non-null  float64
 2   longitude             189986 non-null  float64
 3   property_category     189986 non-null  object 
 4   room_type             189986 non-null  object 
 5   accommodates          189986 non-null  int64  
 6   bathrooms_combined    189986 non-null  float64
 7   beds                  187898 non-null  float64
 8   price                 189986 non-null  float64
 9   number_of_reviews     189986 non-null  int64  
 10  review_scores_rating  147085 non-null  float64
 11  state                 189986 non-null  object 
 12  city                  189986 non-null  object 
dtypes: float64(6), int64(3), object(4)
memory usage: 20.3+ MB


In [ ]:
df_filtered

,id,latitude,longitude,property_category,room_type,accommodates,bathrooms_combined,beds,price,number_of_reviews,review_scores_rating,state,city
0,109,33.983010,-118.386070,condo,Entire home/apt,6,2.0,2.0,129.0,2,4.00,CA,Los Angeles
1,2708,34.096250,-118.346050,other,Private room,1,1.0,1.0,94.0,40,4.85,CA,Los Angeles
2,2732,34.004400,-118.480950,other,Private room,1,1.0,1.0,179.0,24,4.41,CA,Los Angeles
3,2864,33.874900,-118.115150,other,Entire home/apt,2,1.0,1.0,50.0,0,NaN,CA,Los Angeles
4,63416,34.113440,-118.323990,other,Private room,3,1.0,2.0,220.0,161,4.99,CA,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212987,1049006190643672700,47.716741,-122.322171,other,Entire home/apt,12,2.5,8.0,148.0,0,NaN,WA,Seattle
212988,1049036878221888515,47.717079,-122.321981,other,Entire home/apt,6,1.5,3.0,115.0,0,NaN,WA,Seattle
212989,1049307510249446208,47.671660,-122.318940,other,Private room,2,1.0,NaN,42.0,0,NaN,WA,Seattle
212990,1049592710345029420,47.664072,-122.306315,other,Entire home/apt,2,1.0,1.0,64.0,0,NaN,WA,Seattle


In [ ]:
df_filtered.groupby(['state', 'city'])['price'].mean().sort_values(ascending=False)


state  city              
HI     Hawaii                454.426891
CA     San Francisco         392.271530
       Santa Cruz County     324.784701
       Los Angeles           292.551409
TX     Austin                281.062602
CA     San Diego             279.413858
       San Mateo County      231.561042
TN     Nashville             214.554247
CA     Santa Clara County    209.671715
CO     Denver                207.350931
NY     New York City         206.529470
DC     Washington D.C.       193.229973
MA     Cambridge             184.490310
       Boston                183.313262
WA     Seattle               174.158210
IL     Chicago               173.166907
OH     Columbus              142.770066
OR     Salem                 137.854839
NY     Rochester             103.860068
Name: price, dtype: float64

In [ ]:
df_filtered.groupby(['state', 'city'])['price'].min().sort_values(ascending=False)

state  city              
OR     Salem                 30.0
CA     Santa Cruz County     29.0
HI     Hawaii                20.0
MA     Boston                20.0
CO     Denver                17.0
NY     Rochester             16.0
WA     Seattle               13.0
CA     San Francisco         10.0
       San Mateo County      10.0
       Santa Clara County    10.0
DC     Washington D.C.       10.0
TN     Nashville             10.0
MA     Cambridge             10.0
NY     New York City         10.0
OH     Columbus               9.0
IL     Chicago                7.0
CA     Los Angeles            5.0
TX     Austin                 1.0
CA     San Diego              0.0
Name: price, dtype: float64

In [ ]:
def categorize_prices(group):
    labels = ['Low', 'Medium', 'High', 'Very High']
    return pd.qcut(group, q=4, labels=labels, duplicates='drop')

df_filtered['price_category'] = df_filtered.groupby(['state', 'city'])['price'].transform(categorize_prices)

print(df_filtered[['state', 'city', 'price', 'price_category']])

       state         city  price price_category
0         CA  Los Angeles  129.0         Medium
1         CA  Los Angeles   94.0         Medium
2         CA  Los Angeles  179.0           High
3         CA  Los Angeles   50.0            Low
4         CA  Los Angeles  220.0           High
...      ...          ...    ...            ...
212987    WA      Seattle  148.0           High
212988    WA      Seattle  115.0         Medium
212989    WA      Seattle   42.0            Low
212990    WA      Seattle   64.0            Low
212991    WA      Seattle  126.0         Medium

[189986 rows x 4 columns]


In [ ]:
df_filtered[df_filtered['city']=='Los Angeles']

,id,latitude,longitude,property_category,room_type,accommodates,bathrooms_combined,beds,price,number_of_reviews,review_scores_rating,state,city,price_category
0,109,33.983010,-118.386070,condo,Entire home/apt,6,2.0,2.0,129.0,2,4.00,CA,Los Angeles,Medium
1,2708,34.096250,-118.346050,other,Private room,1,1.0,1.0,94.0,40,4.85,CA,Los Angeles,Medium
2,2732,34.004400,-118.480950,other,Private room,1,1.0,1.0,179.0,24,4.41,CA,Los Angeles,High
3,2864,33.874900,-118.115150,other,Entire home/apt,2,1.0,1.0,50.0,0,NaN,CA,Los Angeles,Low
4,63416,34.113440,-118.323990,other,Private room,3,1.0,2.0,220.0,161,4.99,CA,Los Angeles,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45586,1037494589212353675,34.063690,-118.377520,other,Entire home/apt,6,3.0,3.0,140.0,0,NaN,CA,Los Angeles,Medium
45587,1037575019510875191,34.095403,-118.377826,other,Entire home/apt,6,4.0,3.0,936.0,0,NaN,CA,Los Angeles,Very High
45588,1037646556537549700,34.148160,-118.257666,other,Entire home/apt,3,1.0,1.0,144.0,0,NaN,CA,Los Angeles,Medium
45589,1037663899636896646,33.878159,-118.347681,house,Entire home/apt,1,1.0,1.0,95.0,0,NaN,CA,Los Angeles,Medium


In [ ]:
# Group by state and city, then find the min and max price for each state&city combination

price_stats_state_city = df_filtered.groupby(['state', 'city'])['price'].agg(['min', 'max'])

price_stats_state_city

min       max
state city                              
CA    Los Angeles          5.0   99999.0
      San Diego            0.0   99301.0
      San Francisco       10.0   50000.0
      San Mateo County    10.0    8000.0
      Santa Clara County  10.0   39999.0
      Santa Cruz County   29.0    9999.0
CO    Denver              17.0   42770.0
DC    Washington D.C.     10.0   10000.0
HI    Hawaii              20.0   25000.0
IL    Chicago              7.0   99998.0
MA    Boston              20.0   10000.0
      Cambridge           10.0    1500.0
NY    New York City       10.0  100000.0
      Rochester           16.0    1280.0
OH    Columbus             9.0    1673.0
OR    Salem               30.0    1000.0
TN    Nashville           10.0    9999.0
TX    Austin               1.0   38143.0
WA    Seattle             13.0   10000.0

In [ ]:
unique_locations = df_filtered[['state', 'city']].drop_duplicates()

for _, row in unique_locations.iterrows():
    state, city = row['state'], row['city']
    print(f"# Filter for listings in {city}, {state}")

    # Filter for listings in the current city and state
    city_listings = df_filtered[(df_filtered['city'] == city) & (df_filtered['state'] == state)]

    # Group by 'price_category' to find min and max prices
    price_stats = city_listings.groupby('price_category')['price'].agg(['min', 'max'])

    print(price_stats, "\n")


# Filter for listings in Los Angeles, CA
                  min      max
price_category                
Low               5.0     90.0
Medium           91.0    148.0
High            149.0    249.0
Very High       250.0  99999.0 

# Filter for listings in San Diego, CA
                  min      max
price_category                
Low               0.0    110.0
Medium          111.0    166.0
High            167.0    286.0
Very High       287.0  99301.0 

# Filter for listings in San Francisco, CA
                  min      max
price_category                
Low              10.0     95.0
Medium           96.0    145.0
High            146.0    249.0
Very High       250.0  50000.0 

# Filter for listings in San Mateo County, CA
                  min     max
price_category               
Low              10.0    90.0
Medium           91.0   154.0
High            155.0   268.0
Very High       269.0  8000.0 

# Filter for listings in Santa Clara County, CA
                  min      max
price_

In [ ]:
df_filtered

,id,latitude,longitude,property_category,room_type,accommodates,bathrooms_combined,beds,price,number_of_reviews,review_scores_rating,state,city,price_category
0,109,33.983010,-118.386070,condo,Entire home/apt,6,2.0,2.0,129.0,2,4.00,CA,Los Angeles,Medium
1,2708,34.096250,-118.346050,other,Private room,1,1.0,1.0,94.0,40,4.85,CA,Los Angeles,Medium
2,2732,34.004400,-118.480950,other,Private room,1,1.0,1.0,179.0,24,4.41,CA,Los Angeles,High
3,2864,33.874900,-118.115150,other,Entire home/apt,2,1.0,1.0,50.0,0,NaN,CA,Los Angeles,Low
4,63416,34.113440,-118.323990,other,Private room,3,1.0,2.0,220.0,161,4.99,CA,Los Angeles,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212987,1049006190643672700,47.716741,-122.322171,other,Entire home/apt,12,2.5,8.0,148.0,0,NaN,WA,Seattle,High
212988,1049036878221888515,47.717079,-122.321981,other,Entire home/apt,6,1.5,3.0,115.0,0,NaN,WA,Seattle,Medium
212989,1049307510249446208,47.671660,-122.318940,other,Private room,2,1.0,NaN,42.0,0,NaN,WA,Seattle,Low
212990,1049592710345029420,47.664072,-122.306315,other,Entire home/apt,2,1.0,1.0,64.0,0,NaN,WA,Seattle,Low


In [ ]:
df_filtered = df_filtered [['id','latitude','longitude', 'property_category', 'room_type', 'accommodates','bathrooms_combined', 'beds',
                            'number_of_reviews', 'review_scores_rating', 'price_category', 'state', 'city']]

In [ ]:
df_filtered.head()

,id,latitude,longitude,property_category,room_type,accommodates,bathrooms_combined,beds,number_of_reviews,review_scores_rating,price_category,state,city
0,109,33.98301,-118.38607,condo,Entire home/apt,6,2.0,2.0,2,4.00,Medium,CA,Los Angeles
1,2708,34.09625,-118.34605,other,Private room,1,1.0,1.0,40,4.85,Medium,CA,Los Angeles
2,2732,34.00440,-118.48095,other,Private room,1,1.0,1.0,24,4.41,High,CA,Los Angeles
3,2864,33.87490,-118.11515,other,Entire home/apt,2,1.0,1.0,0,NaN,Low,CA,Los Angeles
4,63416,34.11344,-118.32399,other,Private room,3,1.0,2.0,161,4.99,High,CA,Los Angeles


New Feature: Distance to Downtown Calculation

We plan to utilize latitude and longitude coordinates to accurately calculate the distance from each city to its downtown area.

In [ ]:
# all cities' downtowns latitude and longitude coordinates from google maps
df_filtered = df_filtered.copy()

downtown_coords = {
    'Los Angeles': (34.0522, -118.2437),
    'San Diego': (32.7157, -117.1611),
    'San Francisco': (37.7749, -122.4194),
    'San Mateo County': (37.4337, -122.4014),
    'Santa Clara County': (37.3541, -121.9552),
    'Santa Cruz County': (36.9741, -122.0308),
    'Denver': (39.7392, -104.9903),
    'Washington D.C.': (38.9072, -77.0369),
    'Hawaii': (21.3069, -157.8583),
    'Chicago': (41.8781, -87.6298),
    'Boston': (42.3601, -71.0589),
    'Cambridge': (42.3736, -71.1097),
    'New York City': (40.7128, -74.0060),
    'Rochester': (43.1566, -77.6088),
    'Columbus': (39.9612, -82.9988),
    'Salem': (44.9429, -123.0351),
    'Nashville': (36.1627, -86.7816),
    'Austin': (30.2672, -97.7431),
    'Seattle': (47.6062, -122.3321),
}


# Haversine formula function
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Earth's radius in kilometers
    return km

# Calculate distance to downtown for each hotel based on the city
def calculate_distance(row):
    city = row['city']
    if city in downtown_coords:
        poi = downtown_coords[city]
        return haversine(poi[1], poi[0], row['longitude'], row['latitude'])
    else:
        return np.nan

df_filtered['distance_to_downtown_km'] = df_filtered.apply(calculate_distance, axis=1)

# Simple rating system: inverse of distance (for demonstration)
# Adjust as necessary for your criteria
max_distance = df_filtered['distance_to_downtown_km'].max()
df_filtered['location_rating'] = 1 - (df_filtered['distance_to_downtown_km'] / max_distance)

print(df_filtered[['id', 'city', 'distance_to_downtown_km', 'location_rating']])

                         id         city  distance_to_downtown_km  \
0                       109  Los Angeles                15.210789   
1                      2708  Los Angeles                10.623436   
2                      2732  Los Angeles                22.500332   
3                      2864  Los Angeles                23.004930   
4                     63416  Los Angeles                10.052158   
...                     ...          ...                      ...   
212987  1049006190643672700      Seattle                12.314069   
212988  1049036878221888515      Seattle                12.352450   
212989  1049307510249446208      Seattle                 7.345297   
212990  1049592710345029420      Seattle                 6.718827   
212991  1050008701219654211      Seattle                 9.404387   

        location_rating  
0              0.959412  
1              0.971653  
2              0.939961  
3              0.938615  
4              0.973177  
...            

In [ ]:
df_filtered = df_filtered [['id', 'property_category', 'room_type', 'accommodates','bathrooms_combined', 'beds',
                            'number_of_reviews', 'review_scores_rating', 'price_category', 'location_rating','state', 'city']]

In [ ]:
import pandas as pd

# Apply one-hot encoding
df_encoded = pd.get_dummies(df_filtered, columns=['property_category', 'room_type', 'price_category'])

df_encoded.head()

,id,accommodates,bathrooms_combined,beds,number_of_reviews,review_scores_rating,location_rating,state,city,property_category_apartment,property_category_condo,property_category_hotel,property_category_house,property_category_other,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,price_category_Low,price_category_Medium,price_category_High,price_category_Very High
0,109,6,2.0,2.0,2,4.00,0.959412,CA,Los Angeles,0,1,0,0,0,1,0,0,0,0,1,0,0
1,2708,1,1.0,1.0,40,4.85,0.971653,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0
2,2732,1,1.0,1.0,24,4.41,0.939961,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,0,1,0
3,2864,2,1.0,1.0,0,NaN,0.938615,CA,Los Angeles,0,0,0,0,1,1,0,0,0,1,0,0,0
4,63416,3,1.0,2.0,161,4.99,0.973177,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,0,1,0


In [ ]:
df_encoded.shape

(189986, 22)

In [ ]:
df_encoded = df_encoded.dropna()

In [ ]:
df_encoded.shape

(145726, 22)

In [ ]:
df_encoded

,id,accommodates,bathrooms_combined,beds,number_of_reviews,review_scores_rating,location_rating,state,city,property_category_apartment,property_category_condo,property_category_hotel,property_category_house,property_category_other,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,price_category_Low,price_category_Medium,price_category_High,price_category_Very High
0,109,6,2.0,2.0,2,4.00,0.959412,CA,Los Angeles,0,1,0,0,0,1,0,0,0,0,1,0,0
1,2708,1,1.0,1.0,40,4.85,0.971653,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0
2,2732,1,1.0,1.0,24,4.41,0.939961,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,0,1,0
4,63416,3,1.0,2.0,161,4.99,0.973177,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,0,1,0
5,5728,1,1.0,1.0,316,4.78,0.949871,CA,Los Angeles,0,0,0,0,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212902,1036533737039307984,2,1.0,1.0,1,5.00,0.984583,WA,Seattle,0,0,0,0,1,1,0,0,0,1,0,0,0
212904,1036969098338903834,3,1.5,2.0,0,5.00,0.972153,WA,Seattle,0,0,0,0,1,1,0,0,0,0,0,1,0
212908,1038025760131186433,7,3.0,4.0,1,5.00,0.962052,WA,Seattle,0,0,0,0,1,1,0,0,0,0,0,1,0
212923,1040863596595198060,4,2.5,2.0,2,5.00,0.972361,WA,Seattle,0,0,0,0,1,1,0,0,0,0,0,1,0


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Hotel Recommendation").getOrCreate()

# Convert Pandas DataFrame to PySpark DataFrame
df_encoded = spark.createDataFrame(df_encoded)

In [ ]:
df_encoded.show()

+-----+------------+------------------+----+-----------------+--------------------+------------------+-----+-----------+---------------------------+-----------------------+-----------------------+-----------------------+-----------------------+-------------------------+--------------------+----------------------+---------------------+------------------+---------------------+-------------------+------------------------+
|   id|accommodates|bathrooms_combined|beds|number_of_reviews|review_scores_rating|   location_rating|state|       city|property_category_apartment|property_category_condo|property_category_hotel|property_category_house|property_category_other|room_type_Entire home/apt|room_type_Hotel room|room_type_Private room|room_type_Shared room|price_category_Low|price_category_Medium|price_category_High|price_category_Very High|
+-----+------------+------------------+----+-----------------+--------------------+------------------+-----+-----------+---------------------------+------

## Content based recommendation system

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.sql.types import FloatType
from pyspark.ml.linalg import Vectors
import numpy as np

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Hotel Recommendation") \
    .getOrCreate()

# Assuming `df_encoded` is your DataFrame
# Filter the DataFrame based on user input for state and city
user_state = "CA"  # example state
user_city = "Los Angeles"  # example city

filtered_df = df_encoded.filter((col("state") == user_state) & (col("city") == user_city))

# Create feature vectors
feature_cols = [
    "accommodates", "bathrooms_combined", "beds", "number_of_reviews",
    "review_scores_rating", "location_rating", "property_category_apartment",
    "property_category_condo", "property_category_hotel", "property_category_house",
    "property_category_other", "room_type_Entire home/apt", "room_type_Hotel room",
    "room_type_Private room", "room_type_Shared room", "price_category_Low",
    "price_category_Medium", "price_category_High", "price_category_Very High"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
featured_df = assembler.transform(filtered_df)

In [ ]:
# Normalize the feature vectors
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
normalized_df = normalizer.transform(featured_df)

In [ ]:
# Function to calculate cosine similarity (this will be registered as a UDF)
def cosine_similarity(vector1, vector2):
    return float(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

# Register UDF for cosine similarity
cosine_similarity_udf = udf(cosine_similarity, FloatType())

In [ ]:
# Fetch the normalized feature vector of the chosen hotel
hotel_id = 2708
chosen_vector = normalized_df.filter(col("id") == hotel_id).select("normFeatures").first()["normFeatures"]

# Convert chosen_vector to a dense list to be broadcasted
chosen_vector_broadcast = spark.sparkContext.broadcast(chosen_vector.toArray())

# Adding a similarity column based on cosine similarity
@udf(FloatType())
def similarity_udf(features):
    return cosine_similarity(features.toArray(), chosen_vector_broadcast.value)

similarity_df = normalized_df.withColumn("similarity", similarity_udf(col("normFeatures")))

# Show top 10 similar hotels, excluding the chosen one
top10_similar = similarity_df.filter(col("id") != hotel_id).orderBy("similarity", ascending=False).limit(10)
top10_similar.show()

+------------------+------------+------------------+----+-----------------+--------------------+------------------+-----+-----------+---------------------------+-----------------------+-----------------------+-----------------------+-----------------------+-------------------------+--------------------+----------------------+---------------------+------------------+---------------------+-------------------+------------------------+--------------------+--------------------+----------+
|                id|accommodates|bathrooms_combined|beds|number_of_reviews|review_scores_rating|   location_rating|state|       city|property_category_apartment|property_category_condo|property_category_hotel|property_category_house|property_category_other|room_type_Entire home/apt|room_type_Hotel room|room_type_Private room|room_type_Shared room|price_category_Low|price_category_Medium|price_category_High|price_category_Very High|            features|        normFeatures|similarity|
+------------------+--

In [ ]:
top10_similar_pandas_df = top10_similar.toPandas()
top10_similar_pandas_df

,id,accommodates,bathrooms_combined,beds,number_of_reviews,review_scores_rating,location_rating,state,city,property_category_apartment,property_category_condo,property_category_hotel,property_category_house,property_category_other,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,price_category_Low,price_category_Medium,price_category_High,price_category_Very High,features,normFeatures,similarity
0,653238387531934967,1,1.0,1.0,39,4.90,0.939137,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 39.0, 4.9, 0.9391370337996681,...","(0.02538453754550597, 0.02538453754550597, 0.0...",0.999990
1,6234871,1,1.0,1.0,37,4.49,0.941148,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 37.0, 4.49, 0.9411476424460232...","(0.026763946781662898, 0.026763946781662898, 0...",0.999987
2,18225221,1,1.0,1.0,42,4.90,0.942148,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 42.0, 4.9, 0.9421484484263576,...","(0.023603703810450447, 0.023603703810450447, 0...",0.999984
3,8519546,1,1.0,1.0,37,4.68,0.937965,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 37.0, 4.68, 0.9379650378273052...","(0.026747318581445828, 0.026747318581445828, 0...",0.999974
4,8926305,1,1.0,1.0,37,4.81,0.873820,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 37.0, 4.81, 0.8738195453919041...","(0.02673663317475038, 0.02673663317475038, 0.0...",0.999951
5,16918422,1,1.0,1.0,37,4.84,0.920621,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 37.0, 4.84, 0.9206205042186437...","(0.026733064784716782, 0.026733064784716782, 0...",0.999944
6,514836,1,1.0,1.0,44,4.84,0.976562,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 44.0, 4.84, 0.9765615357507746...","(0.02255102862119756, 0.02255102862119756, 0.0...",0.999922
7,825354805491685669,1,1.5,1.0,42,4.83,0.963796,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.5, 1.0, 42.0, 4.83, 0.9637957636672047...","(0.023599692904450405, 0.035399539356675606, 0...",0.999920
8,21231860,1,1.0,1.0,45,4.91,0.922196,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 45.0, 4.91, 0.9221957307827013...","(0.0220542769103715, 0.0220542769103715, 0.022...",0.999899
9,26417647,1,1.0,1.0,36,4.89,0.965544,CA,Los Angeles,0,0,0,0,1,0,0,1,0,0,1,0,0,"(1.0, 1.0, 1.0, 36.0, 4.89, 0.9655440752313329...","(0.027453012392898823, 0.027453012392898823, 0...",0.999873
